In [37]:
import glob
import re
import pickle
import os

from PIL import Image
import cv2
import numpy as np
import pandas as pd
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D
from tensorflow.keras.layers import Activation,Dropout,Flatten,Dense
from tensorflow.keras.utils import to_categorical

In [15]:
IMG_DATA_DIR = '../data/scores/'
LABEL_DATA = '../data/inning.csv'
IMG_WIDTH = 320
IMG_HEIGHT = 90
img_files = glob.glob(IMG_DATA_DIR + '*.jpg')
label = pd.read_csv(LABEL_DATA)

In [16]:
# creare train data
x_train = []
y_train = []
x_test = []
y_test = []
for i, img_file in enumerate(img_files):
    file_num = re.sub(r'\D', '', img_file)
    label_num = label[label['frame'] == int(file_num)]['inning'].values[0]
    img = Image.open(img_file)
    img = img.resize((IMG_WIDTH, IMG_HEIGHT))
    img = np.asarray(img)

    if i % 5 == 0:
        x_test.append(img)
        y_test.append(label_num)
    else:
        x_train.append(img)
        y_train.append(label_num)

x_train = np.array(x_train)
x_test  = np.array(x_test)
y_train = np.array(y_train)
y_test  = np.array(y_test)

In [17]:
# preprocessing
x_train = x_train.astype('float32') / 255
x_test  = x_test.astype('float32') / 255
y_train = to_categorical(y_train, 10)
y_test  = to_categorical(y_test, 10)

In [19]:
# create model
model = Sequential()
model.add(Conv2D(8, kernel_size=(3, 3), activation='relu', input_shape=(IMG_HEIGHT, IMG_WIDTH, 3)))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
model.add(Conv2D(16, kernel_size=(3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(32, activation='relu'))
model.add(Dense(10, activation='softmax'))

model.compile(loss='categorical_crossentropy', optimizer='sgd', metrics=['accuracy'])

In [20]:
history = model.fit(x_train, y_train, epochs=15)

Epoch 1/15


2023-01-11 14:45:01.003727: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


81/81 [==============================] - 4s 35ms/step - loss: 1.8657 - accuracy: 0.3169
Epoch 2/15
81/81 [==============================] - 3s 34ms/step - loss: 1.0532 - accuracy: 0.6602
Epoch 3/15
81/81 [==============================] - 3s 34ms/step - loss: 0.4454 - accuracy: 0.8857
Epoch 4/15
81/81 [==============================] - 3s 33ms/step - loss: 0.1467 - accuracy: 0.9708
Epoch 5/15
81/81 [==============================] - 3s 34ms/step - loss: 0.0532 - accuracy: 0.9914
Epoch 6/15
81/81 [==============================] - 3s 34ms/step - loss: 0.0608 - accuracy: 0.9899
Epoch 7/15
81/81 [==============================] - 3s 33ms/step - loss: 0.0141 - accuracy: 1.0000
Epoch 8/15
81/81 [==============================] - 3s 34ms/step - loss: 0.0097 - accuracy: 1.0000
Epoch 9/15
81/81 [==============================] - 3s 35ms/step - loss: 0.0062 - accuracy: 1.0000
Epoch 10/15
81/81 [==============================] - 3s 34ms/step - loss: 0.0048 - accuracy: 1.0000
Epoch 11/15
81/81 [=

In [21]:
model.evaluate(x_test, y_test)

 4/21 [====>.........................] - ETA: 0s - loss: 0.0031 - accuracy: 1.0000

2023-01-11 14:46:57.048381: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


21/21 [==============================] - 1s 20ms/step - loss: 0.0032 - accuracy: 1.0000


[0.00315684312954545, 1.0]

In [23]:
# save model
pickle.dump(model, open('../models/inning.pkl', 'wb'))

INFO:tensorflow:Assets written to: ram://f1601702-2d52-4d42-bf21-0901136ea265/assets


INFO:tensorflow:Assets written to: ram://f1601702-2d52-4d42-bf21-0901136ea265/assets


モデル読み込み　→ 分類

In [24]:
load_model = pickle.load(open('../models/inning.pkl', 'rb'))

In [36]:
num = 11400
img_file = f'../data/scores416/score{num}.jpg'
tar_img = Image.open(img_file)
tar_img = tar_img.resize((IMG_WIDTH, IMG_HEIGHT))
tar_img = np.asarray(tar_img)
tar_img = tar_img.astype('float32') / 255
# tar_img.shape
load_model.predict(np.array([tar_img])).argmax()

1/1 [==============================] - 0s 26ms/step


5

In [32]:
num = 660
img_file = f'../data/scores/score{num}.jpg'
tar_img = Image.open(img_file)
tar_img = tar_img.resize((IMG_WIDTH, IMG_HEIGHT))
tar_img = np.asarray(tar_img)
tar_img = tar_img.astype('float32') / 255
# tar_img.shape
load_model.predict(np.array([tar_img])).argmax()

1/1 [==============================] - 0s 28ms/step


1

In [40]:
import pandas as pd
num_list = []
for i in range(0, 680000, 60):
    img_file = f'../data/scores416/score{i}.jpg'
    if not os.path.exists(img_file):
        continue
    tar_img = Image.open(img_file)
    tar_img = tar_img.resize((IMG_WIDTH, IMG_HEIGHT))
    tar_img = np.asarray(tar_img)
    tar_img = tar_img.astype('float32') / 255
    pre_num = load_model.predict(np.array([tar_img])).argmax()
    num_list.append([i, pre_num])
df = pd.DataFrame(num_list, columns=['frame', 'inning'])
df.to_csv('../data/inning_416.csv', index=False)

1/1 [==============================] - 0s 11ms/step
